In [1]:
import numpy as np
import seaborn as sns
import tensorflow as tf
from pylab import rcParams
from cumul2norms import NPHC
from scipy.linalg import inv
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from utils.loader import load_data
# Load Cumulants object
kernel = 'exp_d100'
mode = 'nonsym_1'
log10T = 10
url = 'https://s3-eu-west-1.amazonaws.com/nphc-data/{}_{}_log10T{}_with_params_without_N.pkl.gz'.format(kernel, mode, log10T)
cumul, Alpha, Beta, Gamma = load_data(url)

In [88]:
# Params
learning_rate = 1e8
training_epochs = 1000
display_step = 200
d = cumul.dim

In [89]:
_, s, _ = np.linalg.svd(cumul.C)
lbd_max = s[0]
initial = tf.constant(inv(np.eye(d) - cumul.C / (1.1*lbd_max)).astype(np.float32), shape=[d,d])

In [ ]:
list_R = []
for i, alpha in enumerate(np.logspace(0,-4,10)):
    if i == 0: R_init = initial
    R = NPHC(cumul,R_init,alpha=alpha,training_epochs=training_epochs,learning_rate=l_r,display_step=display_step)
    R_init = R.copy()
    list_R.append(R)